<font face= "consolas" size= 5>
        
        A A A     Y Y Y Y Y Y    U U U U   S S S S S   H H H H H
        | | |     | | | | | |    | | | |   | | | | |   | | | | |
        \ | /     \ \ \ / / /    \ \ / /   \ \ | / /   \ \ | / / 
          |        \ \ | / /      \ | /     \ \ / /     \ \ / /
          •         \ \ / /         |        \ | /       \ | /
          A          \ | /          •          |           |
                       |            U          •           •               
                       •                       S           H
                       Y
</font>

Looks like a stream, joining in together to make a beautiful river of Aayush, right?

# 

### Here, in this book
We are going to talk about the WAYS of how groupby function can work.

# 

____

###### 

# <center> *split - apply - combine* </center>

____

# 

There are basically `4` ways to do the groupping:
1. Give a `list or an array` (of same length as df)
2. Give a `column name` (usually a short hand of 1st way)
3. Give a `dict - series` to join by some group - keys
4. Give a `function` that can be applied while groupping

# 

In [3]:
from pandas import DataFrame, Series
from numpy.random import randint

In [4]:
# Making a dataframe
data = dict(
            key1 = list('AABBCAABCC'),
            key2 = ['one', 'two', 'three', 'one', 'three', 'two', 'two', 'three', 'one', 'two'],
            data1 = randint(0, 100, 10),
            data2 = randint(0, 100, 10)
            )

df = DataFrame(data)
df

,key1,key2,data1,data2
0,A,one,80,21
1,A,two,44,83
2,B,three,83,15
3,B,one,31,75
4,C,three,91,79
5,A,two,4,13
6,A,two,74,82
7,B,three,75,16
8,C,one,80,82
9,C,two,89,51


<font face= "product sans" style= "font-size: 150px"> 1. </font> <font size= 6> List or Array </font>

In [5]:
# Making a dataframe
data = dict(
            key1 = list('AABBCAABCC'),
            key2 = ['one', 'two', 'three', 'one', 'three', 'two', 'two', 'three', 'one', 'two'],
            data1 = randint(0, 100, 10),
            data2 = randint(0, 100, 10)
            )

df = DataFrame(data)
df

,key1,key2,data1,data2
0,A,one,75,61
1,A,two,58,97
2,B,three,37,32
3,B,one,28,92
4,C,three,27,66
5,A,two,7,2
6,A,two,37,52
7,B,three,97,15
8,C,one,49,62
9,C,two,41,94


This is an simple and "behind the scenes" kind of approach.

In [6]:
df.groupby(df['key1']).mean()

,data1,data2
key1,,
A,44.25,53.000000
B,54.00,46.333333
C,39.00,74.000000


For multiple keys...

In [7]:
df.groupby([df['key1'], df['key2']]).mean()

data1      data2
key1 key2                   
A    one     75.0  61.000000
     two     34.0  50.333333
B    one     28.0  92.000000
     three   67.0  23.500000
C    one     49.0  62.000000
     three   27.0  66.000000
     two     41.0  94.000000

**And REMEMBER**: The length of the list passed for the group, must be the same as the rows of the df.  
____
### ADVANTAGE:
    The main advantage of this technique is that, we can give custom list to make a group. Like if you want to make group of every 20 rows, you can do this by this.
    
 Example:

There can be EXACT groups but it is possible that the groups may not fit the last all rows. Like number 3 per group will leave single element in the last group if the total elements are 10

In [8]:
import math
n_elems = 3
n_rows = df.shape[0]
n_groups = math.ceil(n_rows / n_elems)

groups = []
for idx in range(n_groups):
    grp = [idx] * n_elems
    groups.extend(grp)

In [9]:
groups

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]

In [10]:
# Making the same length
groups = groups[:n_rows]
groups

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3]

#  

In [11]:
df.groupby(groups).agg(['mean', 'count'])

data1            data2      
        mean count       mean count
0  56.666667     3  63.333333     3
1  20.666667     3  53.333333     3
2  61.000000     3  43.000000     3
3  41.000000     1  94.000000     1

The advantage achieved!  
But the problem is there if we try to use some more numbers to group by - I have posted a question on [stackoverflow](https://stackoverflow.com/questions/67905381/grouping-with-list-in-df-groupby-seems-not-working/67906076#67906076)

# And here is the Answer.

We have to pass an INDEX, instead of list.

```python
# Instead of my algorithm
import math
n_elems = 3
n_rows = df.shape[0]
n_groups = math.ceil(n_rows / n_elems)

groups = []
for idx in range(n_groups):
    grp = [idx] * n_elems
    groups.extend(grp)
    
# I could have done just FREAKIN' in single line!
df.index // n_rows

# ↑ This would give the same result as my algorithm does.
```
Now, there is no problem with my algorithm and of cours use floor division so that we can get the 0, 0, 0, 1, 1, ... 

Now, that doesn't make any difference, but the things is WHAT WE ARE PASSING in the group by.
___
I was passing the list in the gourby call - which was not something correct. If I convert THE SAME list into the pd.Index(list) then it would work! 
So that means that, I have to pass the index object instead of the list to make groups of rows.  
*(It works with Index obj and also np.array)*

But I would encourage you ***not to use*** LIST as the groupper. Instead use either of *numpy array, Series, Index* or other if they are compatible but NOT LIST. Because while giving the list, it tries to find the INDEX in them and will group by that... that is confusing for it. So we will stick with the three ways - array, series, index.

___
Some note on Series:  
I know Series, is one of the techniques for the row groupping... but what it actully does is discussed in the 3. point below. But for some highlight here: Series is a mapping like dict. So it binds the index 0, 1, 2 ... with the groups 0, 0, 0, 1, 1 ... and then does the trick. (If index is different in the series with the df's - then we may need to make the same as both of them - THIS IS NOT HARD. SIMPLE. PAUSE. THINK. ACHIEVE ∞

# 

<font face= "product sans" style= "font-size: 150px"> 2. </font> <font size= 6> Column name only </font>

In [12]:
# Making a dataframe
data = dict(
            key1 = list('AABBCAABCC'),
            key2 = ['one', 'two', 'three', 'one', 'three', 'two', 'two', 'three', 'one', 'two'],
            data1 = randint(0, 100, 10),
            data2 = randint(0, 100, 10)
            )

df = DataFrame(data)
df

,key1,key2,data1,data2
0,A,one,46,68
1,A,two,68,94
2,B,three,73,40
3,B,one,50,90
4,C,three,73,39
5,A,two,40,3
6,A,two,75,57
7,B,three,63,61
8,C,one,37,7
9,C,two,17,63


In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
A,57.250000,55.500000
B,62.000000,63.666667
C,42.333333,36.333333


Here, the key2 columns is excluded. Why? Because it is called ***nuisance column*** which has not any compatible values which can be used for the aggregation we are using.

So in case of `mean()` it won't show up - we can't get mean of strings. But if we do `count` or even `max` then it would appear.

In [14]:
df.groupby('key1').max()

,key2,data1,data2
key1,,,
A,two,75,94
B,three,73,90
C,two,73,63


# 

### <center> Did you know?  </center>
<center> You can <b>iterate over</b> the groups? WITHOUT NEED OF AGG FUNC! </center>

In [15]:
for group in df.groupby('key1'):
    print(group, end= "\n" * 3)

('A',   key1 key2  data1  data2
0    A  one     46     68
1    A  two     68     94
5    A  two     40      3
6    A  two     75     57)


('B',   key1   key2  data1  data2
2    B  three     73     40
3    B    one     50     90
7    B  three     63     61)


('C',   key1   key2  data1  data2
4    C  three     73     39
8    C    one     37      7
9    C    two     17     63)




Cool, huh?

Now, do something like... ↓

In [16]:
dict(list(df.groupby('key1')))

{'A':   key1 key2  data1  data2
 0    A  one     46     68
 1    A  two     68     94
 5    A  two     40      3
 6    A  two     75     57,
 'B':   key1   key2  data1  data2
 2    B  three     73     40
 3    B    one     50     90
 7    B  three     63     61,
 'C':   key1   key2  data1  data2
 4    C  three     73     39
 8    C    one     37      7
 9    C    two     17     63}

In [17]:
dict_of_df = dict(list(df.groupby('key1')))
dict_of_df['A']

,key1,key2,data1,data2
0,A,one,46,68
1,A,two,68,94
5,A,two,40,3
6,A,two,75,57


COOLER, huh?

# 

# Okay, Okay... 
Enough amount of grouping by ROWS. How can we do the same with columns? Hmmm interesting isn't it?

# `.groupby()` with ***axis = 1***

In [18]:
df

,key1,key2,data1,data2
0,A,one,46,68
1,A,two,68,94
2,B,three,73,40
3,B,one,50,90
4,C,three,73,39
5,A,two,40,3
6,A,two,75,57
7,B,three,63,61
8,C,one,37,7
9,C,two,17,63


In [19]:
list(df.groupby('key1', axis= 1))

[]

Hmm... it didn't do anything may be because it woudn't work in this way...

In [20]:
# Trying with the LIST (1.) method
list(df.groupby([0, 0, 1, 1], axis= 1))

[(0,
    key1   key2
  0    A    one
  1    A    two
  2    B  three
  3    B    one
  4    C  three
  5    A    two
  6    A    two
  7    B  three
  8    C    one
  9    C    two),
 (1,
     data1  data2
  0     46     68
  1     68     94
  2     73     40
  3     50     90
  4     73     39
  5     40      3
  6     75     57
  7     63     61
  8     37      7
  9     17     63)]

Worked! 

Look, to group by the columns, we ***might only have*** this lise option / method 1. to do the stuff. 

####  More meaningful... group by columns according to their `dtype`

In [21]:
df.dtypes

key1     object
key2     object
data1     int32
data2     int32
dtype: object

In [22]:
list(df.groupby(df.dtypes, axis= 1))

[(dtype('int32'),
     data1  data2
  0     46     68
  1     68     94
  2     73     40
  3     50     90
  4     73     39
  5     40      3
  6     75     57
  7     63     61
  8     37      7
  9     17     63),
 (dtype('O'),
    key1   key2
  0    A    one
  1    A    two
  2    B  three
  3    B    one
  4    C  three
  5    A    two
  6    A    two
  7    B  three
  8    C    one
  9    C    two)]

#### Nouice!

# 

<font face= "product sans" style= "font-size: 150px"> 3. </font> <font size= 6> Dict or Series (mapping) </font>

**The things is:** I have many friends - but some of them are "Good" and some of them are "Great". Now, if I want to categorize them by their goodness, and want to find some average age, then ... `dicts / series` will help!

In [23]:
my_friends = DataFrame({"age": randint(12, 50, 10),
                        "height": randint(120, 200, 10)},
                       index= ["Sameer", "Mitesh", "Baa", "Muna", "Chachu", "Shah", "Dhabudo", "Mitla", "Muniya", "Smla"])

my_friends.index.name = "friends"
my_friends

,age,height
friends,,
Sameer,37,129
Mitesh,16,130
Baa,17,128
Muna,37,164
Chachu,27,143
Shah,25,157
Dhabudo,41,189
Mitla,29,127
Muniya,24,121


In [24]:
mapping = dict(list(zip(["Sameer", "Mitesh", "Baa", "Muna", "Chachu", "Shah", "Dhabudo", "Mitla", "Muniya", "Smla"],
    ['GOOD', 'GREAT',    'GREAT', 'GOOD', 'GREAT', 'GOOD', 'GOOD', 'GOOD', 'GREAT', 'GREAT', 'GOOD'])))

mapping

{'Sameer': 'GOOD',
 'Mitesh': 'GREAT',
 'Baa': 'GREAT',
 'Muna': 'GOOD',
 'Chachu': 'GREAT',
 'Shah': 'GOOD',
 'Dhabudo': 'GOOD',
 'Mitla': 'GOOD',
 'Muniya': 'GREAT',
 'Smla': 'GREAT'}

Just instead of writing each name... I got them in this way... but you can also do like {"sameer": "good", ...}

In [25]:
my_friends.groupby(mapping)['age'].mean()

GOOD     33.8
GREAT    19.4
Name: age, dtype: float64

In [26]:
my_friends.groupby(mapping)['age'].count()

GOOD     5
GREAT    5
Name: age, dtype: int64

The same technuque would work with axis = 1, but here instead of mapping with the names, you need to map with the columns like {"age": ... }

### And guyz, the same is with series.
SO, that means (as explained above) series can help us to combine row wise instead of using list. 

**That's why series is one of the 3 techniques - numpy, series, index**.

# 

<font face= "product sans" style= "font-size: 150px"> 4. </font> <font size= 6> Function </font> <br>
*This is now amazingly useful!*

**The main Idea**: When we want to group by some charecterestic of index,<br>
***Either***: we can make a column and then group by that <br>
but, <br>
***INSTED***, we can pass the function directly so that we won't need to create a column and do the stuff. So easy.

See, the function will be applied to INDEX (axis 1 / 0).

### I want to group by Length of my friends names...

In [27]:
my_friends.groupby(len).mean()

,age,height
3,17.0,128.000000
4,25.0,167.666667
5,29.0,127.000000
6,26.0,130.750000
7,41.0,189.000000


# 

# → `.groupby( level = 1 )`
This becomes EXTREMELY usful when you have the multiIndex. <br>
*(and of course as we have seen before - mean() like function are also having level argument)*

--<br>
See, that ↑ level parametered group by is helpful **when you want to groupby the INDEX - not the COLUMN** so it is popular to use `df.groupby( level = 0 )` which is exactly equivalent to `df.groupby(df.index)`

# 

# That's it!
In the next book, we will look about the aggfuncs